In [95]:
# Load packages
import os
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np 

In [96]:
home_path = os.getenv('HOME')
osm_building_shp = home_path+'/GeoData/OSM/algeria-latest-free.shp/gis_osm_buildings_a_free_1.shp'
osm_building_gdf = gpd.read_file(osm_building_shp)
osm_building_gdf.head()

,osm_id,code,fclass,name,type,geometry
0,23453590,1500,building,None,None,"POLYGON ((3.28630 36.72337, 3.28680 36.72384, ..."
1,23453621,1500,building,None,industrial,"POLYGON ((3.28839 36.72555, 3.28945 36.72653, ..."
2,23453649,1500,building,None,industrial,"POLYGON ((3.29652 36.73153, 3.29695 36.73203, ..."
3,26224008,1500,building,Hilton Alger,hotel,"POLYGON ((3.15595 36.73836, 3.15634 36.73856, ..."
4,26224288,1500,building,Crystal Lounge,None,"POLYGON ((3.15561 36.73811, 3.15583 36.73823, ..."


In [97]:
# Load the targeted dataset
angem_gdf = gpd.read_file('./angem_to_check.geojson')
angem_gdf.head()

,num,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,geometry
0,1,1,ADRAR,Agence,"CITE 500 VILLAS CNEP N°40, ADRAR",adrar@angem.dz,049 96 76 19,049 96 46 74,ADRAR,27.880138,-0.281887,POINT (-0.28189 27.88014)
1,2,2,CHLEF,Agence,"RUE DJOUBA, HAY BEN SOUNA, CHLEF",chlef@angem.dz,027 79 21 59,027 79 21 58,CHLEF,36.160921,1.315595,POINT (1.31560 36.16092)
2,3,3,LAGHOUAT,Agence,AVENUE 05 JUILLET MAAMOURAH-LAGHOUAT,laghouat@angem.dz,029 13 25 37,029 13 25 38,LAGHOUAT,33.807834,2.862829,POINT (2.86283 33.80783)
3,4,4,OEB,Agence,ANCIENNNE CITE ADMINISTRATIVE BP 2002 Oum El B...,oumelbouaghi@angem.dz,032 56 51 67,032 56 52 43,OEB,35.868879,7.110827,POINT (7.11083 35.86888)
4,5,5,BATNA,Agence,CITE NACER CITE ADMINISTRATIVE BLOC 02 BATNA,batna@angem.dz,033 25 39 79,033 25 39 78,BATNA,35.547137,6.167369,POINT (6.16737 35.54714)


In [98]:
# Define source CRS to geographic and project to a Pseudo-Mercator
# Useful for spatial analysis (buffer, intersect, ...)

angem_gdf = angem_gdf.set_crs("EPSG:4326")
angem_gdf = angem_gdf.to_crs("EPSG:3857")

osm_building_gdf = osm_building_gdf.set_crs("EPSG:4326")
osm_building_gdf = osm_building_gdf.to_crs("EPSG:3857")

osm_building_gdf.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [99]:
# Replace DataFrame geometries with buffers and write it to a file
angem_gdf['buffer'] = angem_gdf.geometry.buffer(500)
buffers = angem_gdf[['num', 'buffer']]
buffers = buffers.rename(columns={'buffer': 'geometry'})
buffers = gpd.GeoDataFrame(buffers)
buffers = buffers.set_crs("EPSG:3857")
gpd.GeoDataFrame(buffers).to_file("buffers.geojson", driver='GeoJSON')
buffers.head()

,num,geometry
0,1,"POLYGON ((-30879.464 3233870.370, -30881.872 3..."
1,2,"POLYGON ((146951.377 4322786.436, 146948.969 4..."
2,3,"POLYGON ((319188.711 4003027.935, 319186.303 4..."
3,4,"POLYGON ((792073.596 4282594.278, 792071.189 4..."
4,5,"POLYGON ((687048.372 4238485.722, 687045.965 4..."


In [100]:
# Search for buildings that intersect the locations buffers
res_intersection = gpd.overlay(buffers, osm_building_gdf, how='intersection')
res_intersection.count()

num         5734
osm_id      5734
code        5734
fclass      5734
name         510
type        2502
geometry    5734
dtype: int64

In [101]:
res_intersection.head()

,num,osm_id,code,fclass,name,type,geometry
0,1,409342237,1500,building,None,None,"POLYGON ((-31394.346 3233371.101, -31413.937 3..."
1,1,432450670,1500,building,C.N.A.S,None,"POLYGON ((-31387.042 3233873.835, -31374.252 3..."
2,1,432450672,1500,building,Centre Pédagogique pour Handicapés,None,"POLYGON ((-31359.958 3233976.285, -31351.699 3..."
3,1,432450674,1500,building,Cité Universitaire,None,"POLYGON ((-31494.165 3234209.172, -31479.950 3..."
4,1,432450675,1500,building,None,None,"POLYGON ((-31517.932 3234219.411, -31507.668 3..."


In [102]:
# Write intersected building into a GeoJSON file
res = res_intersection[['num', 'geometry']]
res.to_file("intersection.geojson", driver='GeoJSON')

In [103]:
# Group results of intersection and add a to_check column (if intersect then doesn't need to check)
res_grouped = res [['num']]
res_grouped = res_grouped.groupby('num').count()
res_grouped['to_check'] = False
res_grouped.count()

to_check    58
dtype: int64

In [116]:
# Merge with DataFrame and set geometry to centroids
angem_to_ckeck_gdf = angem_gdf.merge(res_grouped, left_on='num', right_on='num', how='left')
angem_to_ckeck_gdf.drop(columns=['buffer'], inplace=True)
angem_to_ckeck_gdf.count()

num                        59
code                       59
nom                        59
type                       59
adresses                   59
e-mail                     59
fax                        59
tel                        59
wilayas de rattachement    59
lat                        59
lon                        59
geometry                   59
to_check                   58
dtype: int64

In [120]:
# Update to_check column :if doesn't intersect then we need to check it manually
angem_to_ckeck_gdf['to_check'] = angem_to_ckeck_gdf.apply(lambda row: row.to_check != False, axis=1)
angem_to_ckeck_gdf[angem_to_ckeck_gdf['to_check'] != False]

,num,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,geometry,to_check
45,46,36,EL TARF,Agence,BLOC N°168 À COTE DE L'AGENCE FONCIERE EL TAREF,tarf@angem.dz,038 30 20 22,038 30 21 07,EL TARF,36.757668,8.307634,POINT (924801.620 4405382.417),True


In [123]:
# Reproject to geograĥic and Write to a GeoPackage file
angem_to_ckeck_gdf = angem_to_ckeck_gdf.set_crs("EPSG:3857")
angem_to_ckeck_gdf = angem_to_ckeck_gdf.to_crs("EPSG:4326")
angem_to_ckeck_gdf.to_file("angem_to_check_osm.geojson", driver='GeoJSON')